# Repetitions in Homeric speeches: get data

The DICES search interface is found at: http://dices.ub.uni-rostock.de/app/speeches/search/ 

## TODO

- ✅ get all speeches for Od. and Il. from Dices API
- ✅ filter speeches for relevant tags (giuramenti, istruzioni, preghiere)
- ✅ create an initial dataframe with relevant data
- ✅ add text fetched from Perseus via CTS URN
- ✅ add lemmatised version of text via CLTK
- ✅ save data to JSON lines format suitable for processing with passim

## Dices tags

| Dices tag      | Expansion |
| ----------- | ----------- |
| cha         | Challenge |
| com | Command |
| con | Consolation|
| del | Deliberation|
| des | Desire and Wish|
| exh | Exhortation and Self-Exhortation|
| far | Farewell|
| gre | Greeting and Reception|
| inf | Information and Description|
| ins | Instruction|
| inv | Invitation |
| lam | Lament |
| lau | Praise and Laudation |
| mes | Message|
| nar | Narration|
| ora | Prophecy, Oracular Speech, and Interpretation|
| per | Persuasion|
| pra | Prayer|
| que | Question|
| req | Request|
| res | Reply to Question|
| tau | Taunt|
| thr | Threat|
| und | Undefined|
| vit | Vituperation|
| vow | Promise and Oath|
| war | Warning|

## Retrieving all Homeric speeches from Dices

In [4]:
from dicesapi import DicesAPI
import pandas as pd

# create a connection to DICES
api = DicesAPI(
    dices_api='http://dices.ub.uni-rostock.de/api/',
    logfile='data/log/dices.log'
)

In [3]:
DicesAPI?

Init signature:
DicesAPI(
    dices_api='http://csa20211203-005.uni-rostock.de/api/',
    cts_api='https://scaife-cts.perseus.org/api/cts',
    logfile=None,
    logdetail=2,
    progress_class=None,
)
Docstring:      a connection to the DICES API
Init docstring:
The __init__ function is called when a class is instantiated. 
It initializes the attributes of the class, and it can take arguments that get passed to it by its parent class. 
In this case, we are using the __init__ function to initialize some attributes in our Dices object.

    self: Refer to the object instance (e
    dices_api=DEFAULT_API: Set the default value of the dices api
    cts_api=DEFAULT_CTS: Set the default cts api to use
    logfile=None: Specify a logfile
    logdetail=LOG_MEDDETAIL: Set the detail level of the log
    progress_class=None: Pass a custom progress class to the dices object
:return: Nothing
:doc-author: Trelent
File:           ~/.pyenv/versions/3.10.0/envs/homeric-repetitions/lib/python3.10/site

In [5]:
od_speeches = api.getSpeeches(work_title='Odyssey', progress=True)
print('Got', len(od_speeches), 'speeches')

Got 673 speeches


In [6]:
il_speeches = api.getSpeeches(work_title='Iliad', progress=True)
print('Got', len(il_speeches), 'speeches')

Got 698 speeches


In [7]:
speeches = il_speeches + od_speeches

In [8]:
len(speeches)

1371

In [35]:
speeches[0]._attributes['tags']
speeches[0].author
speeches[0].work
speeches[0].cluster

<SpeechCluster 2019: Odyssey 3.69 ff.>

## Filter speeches

The `tag` information is contained in the `dicesapi.Speech` class, yet a bid hidden. 

In [124]:
def filter_speeches_by_tag(speeches_list, tags_to_keep):
    filtered_speeches = []
    for speech in speeches_list:
        matching_tags = list(set([t['type'] for t in speech._attributes['tags']]) & set(tags_to_keep))
        if len(matching_tags) > 0:
            filtered_speeches.append(speech)
    return filtered_speeches

In [125]:
# ⚠️ check with Ombretta which tags exactly correspond to "discorso di istruzione, discorso di recapito"
speeches_subset = filter_speeches_by_tag(speeches, ['vow', 'ins', 'pra', 'mes', 'nar'])

In [126]:
len(speeches_subset)

221

## Fetch text of speeches from Perseus

In [11]:
from dicesapi.text import CtsAPI
from dicesapi.jupyter import NotebookPBar
cts = CtsAPI()

In [20]:
test = speeches[-1]

In [21]:
test.urn

'urn:cts:greekLit:tlg0012.tlg001.perseus-grc2:1.106-1.120'

In [22]:
cts.getPassage(test).text

'μάντι κακῶν οὐ πώ ποτέ μοι τὸ κρήγυον εἶπας· αἰεί τοι τὰ κάκʼ ἐστὶ φίλα φρεσὶ μαντεύεσθαι, ἐσθλὸν δʼ οὔτέ τί πω εἶπας ἔπος οὔτʼ ἐτέλεσσας· καὶ νῦν ἐν Δαναοῖσι θεοπροπέων ἀγορεύεις ὡς δὴ τοῦδʼ ἕνεκά σφιν ἑκηβόλος ἄλγεα τεύχει, οὕνεκʼ ἐγὼ κούρης Χρυσηΐδος ἀγλάʼ ἄποινα οὐκ ἔθελον δέξασθαι, ἐπεὶ πολὺ βούλομαι αὐτὴν οἴκοι ἔχειν· καὶ γάρ ῥα Κλυταιμνήστρης προβέβουλα κουριδίης ἀλόχου, ἐπεὶ οὔ ἑθέν ἐστι χερείων, οὐ δέμας οὐδὲ φυήν, οὔτʼ ἂρ φρένας οὔτέ τι ἔργα. ἀλλὰ καὶ ὧς ἐθέλω δόμεναι πάλιν εἰ τό γʼ ἄμεινον· βούλομʼ ἐγὼ λαὸν σῶν ἔμμεναι ἢ ἀπολέσθαι· αὐτὰρ ἐμοὶ γέρας αὐτίχʼ ἑτοιμάσατʼ ὄφρα μὴ οἶος Ἀργείων ἀγέραστος ἔω, ἐπεὶ οὐδὲ ἔοικε· λεύσσετε γὰρ τό γε πάντες ὅ μοι γέρας ἔρχεται ἄλλῃ.'

In [10]:
def download_speech_text(speeches):
    '''Download the text of the speeches from CTS server, append to speech objects'''
    pbar = NotebookPBar(max=len(speeches), prefix='Downloading text')

    for s in speeches:
        if not hasattr(s, 'passage') or s.passage is None:
            s.passage = cts.getPassage(s)
        pbar.update()
    return speeches

In [11]:
speeches_subset = speeches
len(speeches_subset)

1371

In [12]:
speeches_subset = download_speech_text(speeches_subset)

In [13]:
speeches_subset[-1].passage.text

'τοιγὰρ ἐγώ τοι, ξεῖνε, μάλʼ ἀτρεκέως ἀγορεύσω. μήτηρ μέν τέ μέ φησι τοῦ ἔμμεναι, αὐτὰρ ἐγώ γε οὐκ οἶδʼ· οὐ γάρ πώ τις ἑὸν γόνον αὐτὸς ἀνέγνω. ὡς δὴ ἐγώ γʼ ὄφελον μάκαρός νύ τευ ἔμμεναι υἱὸς ἀνέρος, ὃν κτεάτεσσιν ἑοῖς ἔπι γῆρας ἔτετμε. νῦν δʼ ὃς ἀποτμότατος γένετο θνητῶν ἀνθρώπων, τοῦ μʼ ἔκ φασι γενέσθαι, ἐπεὶ σύ με τοῦτʼ ἐρεείνεις.'

In [14]:
len(speeches_subset)

1371

## Process speeches with CLTK

In [15]:
def parse_speech_text(speeches):
    '''Run CLTK NLP pipeline to parse all the speeches'''
    
    pbar = NotebookPBar(max=len(speeches), prefix='Running NLP')

    for s in speeches:
        if not hasattr(s, 'passage') or s.passage is None:
            print('no passage:', s)
        elif not hasattr(s.passage, 'cltk') or s.passage.cltk is None:
            s.passage.runCltkPipeline(remove_punct=True)
        pbar.update()
    return speeches



In [16]:
speeches_subset = parse_speech_text(speeches_subset)

no passage: <Speech 931: Odyssey 10.456-10.465>


In [17]:
s = speeches_subset[0]
s.passage.text

'ὢ πόποι αἰγιόχοιο Διὸς τέκος οὐκέτι νῶϊ ὀλλυμένων Δαναῶν κεκαδησόμεθʼ ὑστάτιόν περ; οἵ κεν δὴ κακὸν οἶτον ἀναπλήσαντες ὄλωνται ἀνδρὸς ἑνὸς ῥιπῇ, ὃ δὲ μαίνεται οὐκέτʼ ἀνεκτῶς Ἕκτωρ Πριαμίδης, καὶ δὴ κακὰ πολλὰ ἔοργε.'

In [21]:
# get all pos tags in order to know what to keep and what to exclude
set([str(w.pos) for w in s.passage.cltk for s in speeches_subset])

{'adjective',
 'adverb',
 'coordinating_conjunction',
 'determiner',
 'interjection',
 'noun',
 'numeral',
 'pronoun',
 'subordinating_conjunction',
 'verb'}

In [22]:
keep_pos_tags = ['verb', 'pronoun', 'noun', 'proper_noun', 'adjective', 'adverb']
" ".join([w.lemma for w in s.passage.cltk if str(w.pos) in keep_pos_tags])

'φίλη σύ πρῶτος κιχάνω οὗδος χῶρος χαίρω μή ἐγώ τὶς κακός νόος ἀντιβολέω σύ μέν οὗτος σῴζω δʼ ἐγώ σύ γάρ ἐγώ γε εὔχομαι ὡς θεός σύ φίλος γούνα ἱκάνω ἐγώ οὗτος ἀγορεύω τυμόω ἐγώ οἶδα τίς γῆ τίς δῆμος τίς ἀνήρ ἐγγάγω ἦ πού τὶς νῆσος εὐδείελος ἐγώ ἀκτή κεῖος ἅλς κλίνω ἐριβάλλω ἀπείρω'

## Speeches to Dataframe

In [24]:
len(speeches_subset)

1371

In [25]:
def make_speeches_table(speeches) -> pd.DataFrame:

    keep_pos_tags = ['verb', 'pronoun', 'noun', 'proper_noun', 'adjective', 'adverb']

    return pd.DataFrame(dict(
        speech_id = speech.id,
        group = f"speech-{'-'.join(sorted([tag['type'] for tag in speech._attributes['tags']]))}",
        length = len([w for w in speech.passage.text]) if speech.passage else None,
        passage_urn = speech.urn,
        speaker = speech.getSpkrString(),
        addressee = speech.getAddrString(),
        lemmatised_text = " ".join([w.lemma for w in speech.passage.cltk]) if speech.passage else None,
        lemmatised_filtered_text = " ".join([w.lemma for w in speech.passage.cltk if str(w.pos) in keep_pos_tags]) if speech.passage else None,
        text = speech.passage.text if speech.passage else None,
        label = f"{speech.author.name}, {speech.work.title} {speech.urn.split(':')[-1]}",
        dices_tags = '|'.join(sorted([tag['type'] for tag in speech._attributes['tags']]))
    ) for speech in speeches)

In [26]:
speeches_df = make_speeches_table(speeches_subset)

In [27]:
speeches_df

,speech_id,group,length,passage_urn,speaker,addressee,lemmatised_text,lemmatised_filtered_text,text,label,dices_tags
0,223,speech-que,215.0,urn:cts:greekLit:tlg0012.tlg001.perseus-grc2:8...,Hera,Athena,ὅς πόπος αἴγιοχος Ζεύς τέκος οὐκέτι νῶϊ ὀλλύω ...,ὅς πόπος αἴγιοχος Ζεύς τέκος οὐκέτι νῶϊ ὀλλύω ...,ὢ πόποι αἰγιόχοιο Διὸς τέκος οὐκέτι νῶϊ ὀλλυμέ...,"Homer, Iliad 8.352-8.356",que
1,605,speech-del,463.0,urn:cts:greekLit:tlg0012.tlg001.perseus-grc2:2...,Andromache,servants,δεῦτε δύω ἐγώ ἕπεσθος ὁράω ὅστις ἔργον τύω αἰδ...,ἐγώ ἕπεσθος ὁράω ὅστις ἔργον τύω αἰδοία ἑκυρός...,"δεῦτε δύω μοι ἕπεσθον, ἴδωμʼ ὅτινʼ ἔργα τέτυκτ...","Homer, Iliad 22.450-22.459",del
2,224,speech-del-des-lam,995.0,urn:cts:greekLit:tlg0012.tlg001.perseus-grc2:8...,Athena,Hera,καί λίαν οὗτος γε μένος θυμός τʼ ὀλείσειμι χεί...,λίαν οὗτος γε μένος θυμός ὀλείσειμι χείρ Ἀργεῖ...,καὶ λίην οὗτός γε μένος θυμόν τʼ ὀλέσειε χερσὶ...,"Homer, Iliad 8.358-8.380",del|des|lam
3,606,speech-lam-ora,1691.0,urn:cts:greekLit:tlg0012.tlg001.perseus-grc2:2...,Andromache,Hector,Ἕκτορ ἐγώ δύστηνος εἰ ἆρα γείνω αἶμαι ἀμφότερο...,Ἕκτορ ἐγώ δύστηνος εἰ ἆρα γείνω αἶμαι ἀμφότερο...,Ἕκτορ ἐγὼ δύστηνος· ἰῇ ἄρα γεινόμεθʼ αἴσῃ ἀμφό...,"Homer, Iliad 22.477-22.514",lam|ora
4,225,speech-com,450.0,urn:cts:greekLit:tlg0012.tlg001.perseus-grc2:8...,Zeus,Iris,βάσκυς εἶμι Ἶρος ταχύς πάλιν τρέπω μηδʼ ἐάω ἄν...,βάσκυς εἶμι Ἶρος ταχύς πάλιν τρέπω ἐάω ἔρχομαι...,"βάσκʼ ἴθι Ἶρι ταχεῖα, πάλιν τρέπε μηδʼ ἔα ἄντη...","Homer, Iliad 8.399-8.408",com
...,...,...,...,...,...,...,...,...,...,...,...
1366,46,speech-com-del,322.0,urn:cts:greekLit:tlg0012.tlg001.perseus-grc2:2...,Odysseus,Greeks,δαιμόνις ἀτρέμας ἧμαι καί ἄλλος μῦθος ἀκούω ὁ ...,δαιμόνις ἀτρέμας ἧμαι ἄλλος μῦθος ἀκούω ὁ σύ φ...,"δαιμόνιʼ ἀτρέμας ἧσο καὶ ἄλλων μῦθον ἄκουε, οἳ...","Homer, Iliad 2.200-2.206",com|del
1367,39,speech-del-inf-nar,893.0,urn:cts:greekLit:tlg0012.tlg001.perseus-grc2:2...,Agamemnon,leaders of the Greeks (Greek soldiers),κλύω φίλος θεῖός ἐγώ ἐνύπνιον ἔρχομαι ὄνειρος ...,κλύω φίλος θεῖός ἐγώ ἐνύπνιον ἔρχομαι ὄνειρος ...,κλῦτε φίλοι· θεῖός μοι ἐνύπνιον ἦλθεν ὄνειρος ...,"Homer, Iliad 2.56-2.75",del|inf|nar
1368,45,speech-com-per-war,350.0,urn:cts:greekLit:tlg0012.tlg001.perseus-grc2:2...,Odysseus,leaders of the Greeks (Greek soldiers),δαιμόνις οὐ σύ ἔοικα κακός ὡς δειδίζω ἀλλʼ αὐτ...,δαιμόνις οὐ σύ ἔοικα κακός ὡς δειδίζω αὐτός κά...,"δαιμόνιʼ οὔ σε ἔοικε κακὸν ὣς δειδίσσεσθαι, ἀλ...","Homer, Iliad 2.190-2.197",com|per|war
1369,705,speech-inf-que-res,1514.0,urn:cts:greekLit:tlg0012.tlg002.perseus-grc2:1...,Athena,Telemachus,τοιγάρ ἐγώ σύ οὗτος μαγος ἀτρεκέως ἀγορεύω Μέν...,ἐγώ σύ οὗτος μαγος ἀτρεκέως ἀγορεύω Μέντη Ἀγχι...,τοιγὰρ ἐγώ τοι ταῦτα μάλʼ ἀτρεκέως ἀγορεύσω. Μ...,"Homer, Odyssey 1.179-1.212",inf|que|res


In [33]:
speeches_df.to_pickle('data/homeric_speeches_df.pickle')

## Convert speeches to Passim format

```json
{
    "id": "...",
    "group": "...",
    "text": "...",
}

In [29]:
import json

def speeches_to_passim(speeches_df : pd.DataFrame, json_path : str, lemmatised=False) -> None:

    # transform DataFrame to passim-amenable JSON
    docs = [
        {
            "id" : idx,
            "group": row.group,
            "label": row.label,
            "dices_tags": row.dices_tags,
            "text": row.lemmatised_filtered_text if lemmatised else row.text,
            "raw_text": row.text if lemmatised else None,
            "passage_urn": row.passage_urn
        }
        for idx, row in speeches_df.iterrows()
    ]

    # write passim docs to JSON file
    with open(json_path, 'w', encoding='utf-8') as f:
        for d in docs:
            print(json.dumps(d, ensure_ascii=False), file=f)



In [30]:
# speeches_to_passim(speeches_df, 'test_input.json')

In [31]:
speeches_to_passim(speeches_df[~speeches_df.text.isna()], 'data/input/homeric_speeches_lemmatised.json', lemmatised=True)

In [32]:
speeches_to_passim(speeches_df[~speeches_df.text.isna()], 'data/input/homeric_speeches_raw.json', lemmatised=False)

## Run Passim

In [161]:
passim_output_path = "data/passim/out_cluster/"
passim_logfile = "data/log/passim.log"

In [162]:
!rm -r {passim_output_path}

In [124]:
# -a : Minimum length of alignment (default: 50)
!seriatim -a 5 {passim_input_path} {passim_output_path} >& {passim_logfile}

^C


## Read Passim's output

In [7]:
import glob, itertools, json
import pandas as pd

# Read one JSON record per line
def read_jsonl_file(f):
  res = []
  for line in f:
    res.append(json.loads(line))
  return res

def read_jsonl(d):
  return list(itertools.chain.from_iterable([read_jsonl_file(open(f)) for f in glob.glob(d + '/*.json')]))

In [4]:
def passim_output_to_dataframe(output_path, dataframe_path, columns_to_keep=['cluster', 'id', 'label', 'dices_tags', 'text', 'raw_text']) -> pd.DataFrame:
    tr_clusters = pd.DataFrame(read_jsonl(output_path))
    print(f'There are {tr_clusters.cluster.unique().size} text reuse clusters in {output_path}')
    output_df = tr_clusters[columns_to_keep]
    output_df.to_csv(dataframe_path)
    return output_df

## Experiments

| Experiment ID     | Passim parameters | Explanation  | \# of extracted clusters |
| ----------- | ----------- |----------|------------|
| `exp0`      | `-n 1 --min-match 1 -a 5` | reused passages consist of at least 1 shared n-grams of size 1 (uni-gram) | 1066 |
| `exp1`| `-n 3 --max-repeat 100`|reused passages consist of at least 5 shared n-grams of size 3 (uni-gram)|23|
| `exp2`|`-n 3 --min-match 3 --max-repeat 100 -w 1`|reused passages consist of at least 3 shared n-grams of size 3 (tri-grams)|24|
| `exp3`|`-n 3 --min-match 3 --max-repeat 100 -a 10`|reused passages consist of at least 3 shared n-grams of size 3, and the aligned passage should be at least 10 characters long (default is `20`)|41|
| `exp4`|`-n 2 --min-match 2 --max-repeat 100 -a 10`|reused passages consist of at least 2 shared n-grams of size 2 (bi-grams), and the aligned passage should be at least 10 characters long (default is `20`)|212|
| `exp5`|`-n 3 --min-match 1 --max-repeat 100 -a 10`|reused passages consist of at least 1 shared n-grams of size 3 (tri-grams), and the aligned passage should be at least 10 characters long (default is `20`)|41|
| `exp6`|`-n 4 --min-match 1 --max-repeat 100 -a 10`|reused passages consist of at least 1 shared n-grams of size 4 (bi-grams), and the aligned passage should be at least 10 characters long (default is `20`)|36|

### Experiment 0 (exp0)

In [205]:
!cat data/passim/exp0/memo.txt

/home/romanell/Documents/impresso/passim/bin/passim -n 1 --min-match 1 -a 5 --max-repeat 100 -w 1 data/input/input_lemmatised.json data/passim/out_cluster


In [207]:
output_path = 'data/passim/exp0/out.json/'
tsv_path = 'data/output/passim_clusters_exp0.csv'
tr_clusters = passim_output_to_dataframe(output_path, tsv_path)

There are 1066 text reuse clusters in data/passim/exp0/out.json/


In [208]:
tr_clusters

,cluster,id,label,dices_tags,text,raw_text
0,0,0,"Homer, Iliad 1.37-1.42",pra|req,κλίνω ἐγώ ὅς Χρύση ἀμφιβαίνω Κίλλας Τενέδοιος ...,"κλῦθί μευ ἀργυρότοξʼ, ὃς Χρύσην ἀμφιβέβηκας Κί..."
1,0,1,"Homer, Iliad 10.278-10.282",pra|req,κλίνω ἐγώ αἴγιοχος Ζεύς τέκος ὅς ἐγώ πόνος παρ...,"κλῦθί μευ αἰγιόχοιο Διὸς τέκος, ἥ τέ μοι αἰεὶ ..."
2,0,2,"Homer, Iliad 10.284-10.294",pra|req,κλύω ἐγώ Ζεύς τέκος Ἀτρυτώνη σπεῖον ἐγώ πατήρ ...,κέκλυθι νῦν καὶ ἐμεῖο Διὸς τέκος Ἀτρυτώνη· σπε...
3,0,4,"Homer, Odyssey 1.32-1.43",lam|nar,ὅς πόπος οἷος νυ θεός βροτοί αἰτιόω ἡμεῖς φημί...,"ὢ πόποι, οἷον δή νυ θεοὺς βροτοὶ αἰτιόωνται· ἐ..."
4,0,6,"Homer, Odyssey 1.158-1.177",nar|que,φίλη ἐγώ νεμεσέω ὅστις λέγω μέλει κίθαρις ἀοιδ...,"ξεῖνε φίλʼ, ἦ καί μοι νεμεσήσεαι ὅττι κεν εἴπω..."
...,...,...,...,...,...,...
3777,8589936020,123,"Homer, Odyssey 3.14-3.20",ins,σύ χρή ἐς αἰδώς ἀβαιρέω τοὔνω πόντος ἐπέπλως ὄ...,"Τηλέμαχʼ, οὐ μέν σε χρὴ ἔτʼ αἰδοῦς, οὐδʼ ἠβαιό..."
3778,8589936020,180,"Homer, Odyssey 17.124-17.146",inf|nar,σύ ἐγώ κρύπτω ἔπος ἐπίκειμαι μιν ὅς γι νῆσος ὁ...,"ὢ πόποι, ἦ μάλα δὴ κρατερόφρονος ἀνδρὸς ἐν εὐν..."
3779,8589936118,109,"Homer, Odyssey 14.462-14.506",nar,ἄγω ἀρτύνω ἡγέομαι Ὀδυσεύς Ἀτρεΐδης Μενέλαος ὁ...,"κέκλυθι νῦν, Εὔμαιε καὶ ἄλλοι πάντες ἑταῖροι, ..."
3780,8589936118,145,"Homer, Iliad 19.155-19.183",com|del|ins,ἄγω λαός σκεδάζω δεῖπνον ἀνόχω ἕπλω δῶρον ἄναξ...,"μὴ δʼ οὕτως, ἀγαθός περ ἐών, θεοείκελʼ Ἀχιλλεῦ..."


### Experiment 1 (exp1)

In [183]:
!cat data/passim/exp1/memo.txt

/home/romanell/Documents/impresso/passim/bin/passim -n 3 --max-repeat 100 data/input/input_lemmatised.json data/passim/exp1


In [212]:
output_path = 'data/passim/exp1/out.json/'
tsv_path = 'data/output/passim_clusters_exp1.csv'
tr_clusters_exp1 = passim_output_to_dataframe(output_path, tsv_path)

There are 23 text reuse clusters in data/passim/exp1/out.json/


### Experiment 2 (exp2)

In [191]:
!cat data/passim/exp2/memo.txt

/home/romanell/Documents/impresso/passim/bin/passim -n 3 --min-match 3 --max-repeat 100 -w 1 data/input/input_lemmatised.json data/passim/exp2


In [213]:
output_path = 'data/passim/exp2/out.json/'
tsv_path = 'data/output/passim_clusters_exp2.csv'
tr_clusters_exp2 = passim_output_to_dataframe(output_path, tsv_path)

There are 24 text reuse clusters in data/passim/exp2/out.json/


### Experiment 3 (exp3)

In [192]:
!cat data/passim/exp3/memo.txt

/home/romanell/Documents/impresso/passim/bin/passim -n 3 --min-match 3 --max-repeat 100 -a 10 data/input/input_lemmatised.json data/passim/exp3



In [214]:
output_path = 'data/passim/exp3/out.json/'
tsv_path = 'data/output/passim_clusters_exp3.csv'
tr_clusters_exp3 = passim_output_to_dataframe(output_path, tsv_path)

There are 41 text reuse clusters in data/passim/exp3/out.json/


### Experiment 4 (exp 4)

In [202]:
!cat data/passim/exp4/memo.txt

/home/romanell/Documents/impresso/passim/bin/passim -n 2 --min-match 2 --max-repeat 100 -a 10 data/input/input_lemmatised.json data/passim/exp4


In [210]:
output_path = 'data/passim/exp4/out.json/'
tsv_path = 'data/output/passim_clusters_exp4.csv'
tr_clusters_exp4 = passim_output_to_dataframe(output_path, tsv_path)

There are 212 text reuse clusters in data/passim/exp4/out.json/


### Experiment 5 (exp5)

In [8]:
!cat data/passim/exp5/memo.txt

/home/romanell/Documents/impresso/passim/bin/passim -n 3 --min-match 1 --max-repeat 100 -a 10 data/input/input_lemmatised.json data/passim/exp5



In [9]:
output_path = 'data/passim/exp5/out.json/'
tsv_path = 'data/output/passim_clusters_exp5.csv'
tr_clusters_exp5 = passim_output_to_dataframe(output_path, tsv_path)

There are 41 text reuse clusters in data/passim/exp5/out.json/


## Experiment 5 (exp5)

In [11]:
!cat data/passim/exp6/memo.txt

/home/romanell/Documents/impresso/passim/bin/passim -n 4 --min-match 1 --max-repeat 100 -a 10 data/input/input_lemmatised.json data/passim/exp6



In [10]:
output_path = 'data/passim/exp6/out.json/'
tsv_path = 'data/output/passim_clusters_exp6.csv'
tr_clusters_exp6 = passim_output_to_dataframe(output_path, tsv_path)

There are 36 text reuse clusters in data/passim/exp6/out.json/
